In [8]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head() 

In [ ]:
movies.keywords[0]

In [11]:
credits = pd.read_csv('tmdb_5000_credits.csv')


In [ ]:
credits.head(1)['cast'].values

In [ ]:
credits.iloc[0]

In [ ]:
credits.head(1)['crew'].values

In [15]:
# merging data frames
movies = movies.merge(credits, on='title')

In [ ]:
movies.columns

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords', 'cast', 'crew']]
movies.sample(10)

## Preprocessing

In [ ]:
movies.isnull().sum()

In [19]:
movies.dropna(inplace=True)

In [ ]:
movies.isna().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [23]:
# need to get the values of the name key in list but it is in form of a string
import ast

In [24]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
# Test case
convert('[{"id": 28, "name": "Action"}]')

In [ ]:
movies.genres = movies.genres.apply(convert)
movies

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies

In [ ]:
movies.iloc[0].cast

In [30]:
def convert_cast(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter +=1
        else:
            break
    return L

In [ ]:
# Top 3 cast
movies['cast'] = movies['cast'].apply(convert_cast)
movies['cast']

In [ ]:
movies.iloc[0]

In [ ]:
movies.iloc[0].crew

In [34]:
def get_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies.crew = movies.crew.apply(get_director)
movies.crew

In [ ]:
movies.head(1)

In [ ]:
movies.overview

In [38]:
movies.overview = movies.overview.apply(lambda x:x.split())

In [ ]:
movies.overview[0]

In [ ]:
movies.head()

In [41]:
movies.genres = movies.genres.apply(lambda x:[i.replace(" ","") for i in x])
movies.keywords = movies.keywords.apply(lambda x:[i.replace(" ","") for i in x])
movies.cast = movies.cast.apply(lambda x:[i.replace(" ","") for i in x])
movies.crew = movies.crew.apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies.overview+movies.genres+movies.keywords+movies.cast+movies.crew
print(movies.iloc[0].tags)

In [ ]:
movies.columns

In [ ]:
new_df = movies[['movie_id','title','tags']]
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df

In [ ]:
new_df.iloc[0].tags

In [ ]:
new_df.tags = new_df.tags.apply(lambda x:x.lower())

In [ ]:
new_df.tags[0]

In [ ]:
new_df

## Vectorization

In [51]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english',binary=True)

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors.shape

In [ ]:
vectors[0].sum()

In [ ]:
cv.get_feature_names_out()

## Stemming

In [ ]:
import nltk
from nltk.corpus import stopwords

stopwords.words('english')


In [57]:
import nltk

In [58]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
new_df['tags'][0]

In [60]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# stem test case
stem('in the 22nd century, a paraplegic marine is')

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df['tags'][0]

## Similarity Check

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)
similarity.shape

In [ ]:
# need to get top 5 hight similarity scores we can sort in descending order

sorted(similarity[0],reverse = True)

# this will be sort the series but index position will also moved we will not only sort but also generate new indices using enemurate

In [ ]:
# this will sort in new indices
# sorted(list(enumerate(similarity[0])))
# this will held the indices also
sorted(list(enumerate(similarity[0])),reverse=True, key= lambda x: x[1])[1:10]

In [ ]:
# getting index of any movie
new_df[new_df['title']=='Life of Pi'].index[0]

In [ ]:
# getting title of any movie using index
new_df.loc[216,'title']

In [70]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True, key= lambda x: x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])
     

In [ ]:
recommend('The Dark Knight Rises')

In [ ]:
recommend('Fight Club')

In [ ]:
recommend('Forrest Gump')

In [ ]:
recommend('Good Will Hunting')